In [ ]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

'''
Author: Enrico Ceccolini
    TODO write the description
'''

#datadir = "/datasets/eurora_data/db1/"
datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"

infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-06-30')
end_time = pd.to_datetime('2014-11-01')

#infile_large_jobs = datadir + "CPUs/WholeData/large_jobs_real_pow_final.csv"
#outfile_large_jobs = datadir + "CPUs/WholeData/large_jobs_real_pow_final.csv"

infile_jobs = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
outfile_jobs = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_res"

In [ ]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])
merged_jobs_to_nodes.head(1)

In [ ]:
i = 0
count = 0
for index, row in jobs.iterrows():
    
    i += 1
    if (row['node_req'] == 0):
        count += 1
        job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == row['job_id_string']]
        nnodes = job_to_nodes.shape[0]
        ncpus = job_to_nodes['ncpus'].sum()
        ngpus = job_to_nodes['ngpus'].sum()
        nmics = job_to_nodes['nmics'].sum()
        mem_requested = job_to_nodes['mem_requested'].sum()

        #print(ncpus)
        jobs.at[index, 'node_req'] = nnodes
        jobs.at[index, 'cpu_req'] = ncpus
        jobs.at[index, 'gpu_req'] = ngpus
        jobs.at[index, 'mic_req'] = nmics
        jobs.at[index, 'mem_req'] = mem_requested
    if (i%10000 == 0):
        jobs.to_csv(outfile_jobs + ".csv")
    if (count % 1000 == 0):
        print(count)
        print("{}/{}".format(i, jobs.shape[0]))
        
print(count)
jobs.to_csv(outfile_jobs + ".csv")


In [ ]:
jobs[jobs['node_req'] == 0]

In [ ]:
jobs.to_csv(outfile_jobs + ".csv")